### Imports

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from math import pi
import plotly.express as px

CSV loading

In [2]:
economy = pd.read_csv("../data/economy_data.csv")
communications = pd.read_csv("../data/communications_data.csv")
demographics = pd.read_csv("../data/demographics_data.csv")
energy = pd.read_csv("../data/energy_data.csv")
geography = pd.read_csv("../data/geography_data.csv")
governmentcivics = pd.read_csv("../data/government_and_civics_data.csv")
transportation = pd.read_csv("../data/transportation_data.csv")

Cleaning and merging

In [3]:
import pandas as pd
import numpy as np

# ------------------------------------------------------------
# Helper: clean numeric-looking columns (remove commas, %, etc.)
# ------------------------------------------------------------
def clean_numeric(series: pd.Series) -> pd.Series:
    """
    Take a pandas Series that should be numeric (but may contain commas,
    percent signs, 'NA', etc.) and return a Series of cleaned strings.
    """
    return (
        series.astype(str)
        .str.replace(",", "", regex=False)
        .str.replace("%", "", regex=False)
        .str.replace("−", "-", regex=False)  # sometimes minus is a different unicode char
        .replace({"nan": np.nan, "NA": np.nan, "": np.nan})
    )

# ------------------------------------------------------------
# 1) Load all CSVs
#    (change to "../data/..." if your notebook is in /notebooks)
# ------------------------------------------------------------

economy = pd.read_csv("../data/economy_data.csv")
comms = pd.read_csv("../data/communications_data.csv")
demo = pd.read_csv("../data/demographics_data.csv")
energy = pd.read_csv("../data/energy_data.csv")
geo = pd.read_csv("../data/geography_data.csv")
gov = pd.read_csv("../data/government_and_civics_data.csv")
trans = pd.read_csv("../data/transportation_data.csv")

# ------------------------------------------------------------
# 2) Define which columns are numeric in each table
#    (so we don't destroy text like Government_Type, Capital, etc.)
# ------------------------------------------------------------

eco_num_cols = [
    "Real_GDP_PPP_billion_USD",
    "GDP_Official_Exchange_Rate_billion_USD",
    "Real_GDP_Growth_Rate_percent",
    "Real_GDP_per_Capita_USD",
    "Unemployment_Rate_percent",
    "Youth_Unemployment_Rate_percent",
    "Budget_billion_USD",
    "Budget_Surplus_billion_USD",
    "Budget_Deficit_percent_of_GDP",
    "Public_Debt_percent_of_GDP",
    "Exports_billion_USD",
    "Imports_billion_USD",
    "Exchange_Rate_per_USD",
    "Population_Below_Poverty_Line_percent",
]

demo_num_cols = [c for c in demo.columns if c != "Country"]

energy_num_cols = [
    "electricity_access_percent",
    "electricity_generating_capacity_kW",
    "coal_metric_tons",
    "petroleum_bbl_per_day",
    "refined_petroleum_products_bbl_per_day",
    "refined_petroleum_exports_bbl_per_day",
    "refined_petroleum_imports_bbl_per_day",
    "natural_gas_cubic_meters",
    "carbon_dioxide_emissions_Mt",
]

trans_num_cols = [
    "airports_paved_runways_count",
    "airports_unpaved_runways_count",
    "heliports_count",
    "roadways_km",
    "railways_km",
    "waterways_km",
    "gas_pipelines_km",
    "oil_pipelines_km",
    "refined_products_pipelines_km",
    "water_pipelines_km",
]

comms_num_cols = [
    "telephone_fixed_subscriptions_total",
    "mobile_cellular_subscriptions_total",
    "internet_users_total",
    "broadband_fixed_subscriptions_total",
]

# In geography, some columns are textual (Land_Boundaries, Coastline),
# others are numeric:
geo_num_cols = [
    "Area_Total",
    "Land_Area",
    "Water_Area",
    "Highest_Elevation",
    "Lowest_Elevation",
    "Forest_Land",
    "Other_Land",
    "Agricultural_Land",
    "Arable_Land (%% of Total Agricultural Land)",
    "Permanent_Crops (%% of Total Agricultural Land)",
    "Permanent_Pasture (%% of Total Agricultural Land)",
    "Irrigated_Land",
]

gov_num_cols = ["Suffrage_Age"]

# ------------------------------------------------------------
# 3) Clean numeric columns in each dataframe
#    (we keep all rows; we just ensure types are clean)
# ------------------------------------------------------------

for col in eco_num_cols:
    economy[col] = pd.to_numeric(clean_numeric(economy[col]), errors="coerce")

for col in demo_num_cols:
    if col != "Country":
        demo[col] = pd.to_numeric(clean_numeric(demo[col]), errors="coerce")

for col in energy_num_cols:
    energy[col] = pd.to_numeric(clean_numeric(energy[col]), errors="coerce")

for col in trans_num_cols:
    trans[col] = pd.to_numeric(clean_numeric(trans[col]), errors="coerce")

for col in comms_num_cols:
    comms[col] = pd.to_numeric(clean_numeric(comms[col]), errors="coerce")

for col in geo_num_cols:
    geo[col] = pd.to_numeric(clean_numeric(geo[col]), errors="coerce")

for col in gov_num_cols:
    gov[col] = pd.to_numeric(clean_numeric(gov[col]), errors="coerce")

# ------------------------------------------------------------
# 4) Merge all tables on Country
#    Use OUTER joins so we NEVER drop any country.
# ------------------------------------------------------------

df = economy.copy()

df = df.merge(demo,   on="Country", how="outer")
df = df.merge(energy, on="Country", how="outer")
df = df.merge(trans,  on="Country", how="outer")
df = df.merge(comms,  on="Country", how="outer")
df = df.merge(geo,    on="Country", how="outer")
df = df.merge(gov,    on="Country", how="outer")

# ------------------------------------------------------------
# 5) Fill missing numeric values with 0.0
#    Justification:
#    - For an interactive dashboard, it's often more practical
#      to treat missing infrastructure quantities as 0 in
#      charts (no errors, clear "no data / no infrastructure").
#    - If later you need to distinguish true zero vs missing,
#      you can keep a copy of the raw merged file or use masks.
# ------------------------------------------------------------

numeric_cols = df.select_dtypes(include=["number"]).columns
df[numeric_cols] = df[numeric_cols].fillna(0.0)

# (Optional) If you prefer to keep NaN and only fill in derived metrics,
# comment out the two lines above.

# ------------------------------------------------------------
# 6) Save the merged, cleaned dataset
# ------------------------------------------------------------

df.to_csv("../data/merged_cia_countries.csv", index=False)
print("Merged data shape:", df.shape)
print("Saved to data/merged_cia_countries.csv")


Merged data shape: (259, 72)
Saved to data/merged_cia_countries.csv


In [6]:
print("Total rows:", len(df))
print("Non-null GDP rows:", df["Real_GDP_per_Capita_USD"].notna().sum())
print("Positive GDP rows:", (df["Real_GDP_per_Capita_USD"] > 0).sum())
print("Min GDP:", df["Real_GDP_per_Capita_USD"].min())
print("Max GDP:", df["Real_GDP_per_Capita_USD"].max())

Total rows: 259
Non-null GDP rows: 259
Positive GDP rows: 220
Min GDP: 0.0
Max GDP: 115700.0


Starting plotting for dashboard, as well as some data checks and investigation

In [ ]:
def create_gdp_histogram(df):
    # Keep only non-null values
    d = df[df["Real_GDP_per_Capita_USD"].notna()].copy()

    fig = px.histogram(
        d,
        x="Real_GDP_per_Capita_USD",
        nbins=40,
        labels={"Real_GDP_per_Capita_USD": "GDP per capita (USD)"},
        hover_data=["Country"],
    )

    # Linear scale (safe, everything shows)
    fig.update_xaxes(title="GDP per capita (USD)")

    # Style for dark dashboard
    fig.update_traces(marker=dict(color="#00CC96"))  # clearly visible green
    fig.update_layout(
        title="Global distribution of GDP per capita",
        bargap=0.05,
        paper_bgcolor="#111111",
        plot_bgcolor="#111111",
        font_color="#FFFFFF",
        margin=dict(l=40, r=20, t=60, b=40),
    )

    return fig

gdp_hist = create_gdp_histogram(df)
gdp_hist.show()

In [4]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("../data/merged_cia_countries.csv")

d = df[df["Real_GDP_per_Capita_USD"].notna() & (df["Real_GDP_per_Capita_USD"] > 0)].copy()

# 1) Make bins (choose either fixed bins or quantiles)
bins = [0, 1000, 5000, 10000, 20000, 40000, 80000, 200000]
d["gdp_bin"] = pd.cut(d["Real_GDP_per_Capita_USD"], bins=bins, right=False)

# 2) Aggregate: count + list of countries
agg = (d.groupby("gdp_bin", dropna=False)
         .agg(
             count=("Country", "size"),
             countries=("Country", lambda s: ", ".join(sorted(s.tolist())))
         )
         .reset_index())

# (Optional) avoid insane tooltips: show only first N countries
N = 25
agg["countries_hover"] = agg["countries"].apply(
    lambda x: x if len(x.split(", ")) <= N else ", ".join(x.split(", ")[:N]) + " …"
)

# 3) Plot bars with custom hover text
fig = px.bar(
    agg,
    x="gdp_bin",
    y="count",
    labels={"gdp_bin": "GDP per capita bin", "count": "# countries"},
    title="Countries per GDP per capita bin",
)

fig.update_traces(
    hovertemplate="<b>%{x}</b><br># countries: %{y}<br><br>%{customdata}<extra></extra>",
    customdata=agg["countries_hover"],
)

fig.show()


C:\Users\20232726\AppData\Local\Temp\ipykernel_26224\653743506.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  agg = (d.groupby("gdp_bin", dropna=False)


TypeError: Object of type Interval is not JSON serializable